In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RepeatedKFold, StratifiedShuffleSplit
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import shap

# 设置全局字体为Arial，并调整大小
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 36
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.titlesize'] = 44
plt.rcParams['xtick.labelsize'] = 36
plt.rcParams['ytick.labelsize'] = 36
plt.rcParams['legend.fontsize'] = 36
plt.rcParams['figure.titlesize'] = 44

# 定义浅色调色板
colors = {
    'RandomForest': ('#4daf4a', '#006400'),
    'XGBoost': ('#377eb8', '#00468B'),
    'ElasticNet': ('#e41a1c', '#990000'),
    'SVR': ('#984ea3', '#5f0080'),
    'GradientBoosting': ('#ff7f00', '#b35900'),
}

# 自定义特征选择器
class CustomFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=10):
        self.k = k
        self.selector = SelectKBest(mutual_info_regression, k=self.k)
    
    def fit(self, X, y):
        self.selector.fit(X, y)
        return self
    
    def transform(self, X):
        return self.selector.transform(X)

# 读取Excel文件
excel_path = r"D:\张雨林\电子科大合作文章\力学三步筛选后特征\力学特征建模.xlsx"
output_folder = os.path.dirname(excel_path)
df = pd.read_excel(excel_path, sheet_name="Sheet1", header=0)

def plot_prediction(y_train, y_train_pred, y_test, y_test_pred, name, r2_train, r2_test):
    plt.figure(figsize=(12, 10))
    
    plt.scatter(y_train, y_train_pred, color=colors[name][0], alpha=0.3, label='Training Set', s=120)
    plt.scatter(y_test, y_test_pred, color=colors[name][1], alpha=0.6, label='Test Set', s=120)
    
    min_val = min(y_train.min(), y_test.min())
    max_val = max(y_train.max(), y_test.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=3)
    
    plt.xlabel('Observed Values', fontsize=40)
    plt.ylabel('Predicted Values', fontsize=40)
    
    plt.text(0.05, 0.95, f'Train R² = {r2_train:.3f}\nTest R² = {r2_test:.3f}', 
             transform=plt.gca().transAxes, fontsize=36, 
             verticalalignment='top', fontweight='bold')
    
    plt.legend(loc='lower right')
    
    plt.gca().set_facecolor('#f0f0f0')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.title(name, fontsize=44, fontweight='bold', pad=20)
    
    plt.tick_params(axis='both', which='major', labelsize=36)
    
    plt.tight_layout()
    output_path = os.path.join(output_folder, f'{name}_comparison_plot.png')
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    print(f"Saved plot for {name} at {output_path}")
    plt.close()
  
def plot_feature_importance(importance_df, model_name):
    plt.figure(figsize=(14, 10))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(6), 
                palette=[colors[model_name][0], colors[model_name][1]], edgecolor='black')
    plt.title(f'Top 6 Features - {model_name}', fontweight='bold', fontsize=44)
    plt.xlabel('Importance Score', fontweight='bold', fontsize=40)
    plt.ylabel('', fontweight='bold')
    plt.tick_params(axis='both', which='major', labelsize=36)
    plt.tight_layout()
    output_path = os.path.join(output_folder, f'{model_name}_Feature_Importance.png')
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    print(f"Saved feature importance plot at {output_path}")
    plt.close()

# 修改相关性热图绘图函数
def plot_correlation_heatmap(corr_matrix, model_name):
    plt.figure(figsize=(14, 12))
    sns.heatmap(corr_matrix, annot=True, cmap='PuOr', vmin=-1, vmax=1, center=0, 
                square=True, linewidths=0.5, cbar_kws={"shrink": .8}, fmt='.2f',
                annot_kws={"size": 30})
    plt.title(f'Correlation Heatmap of Top 6 Features - {model_name}', fontweight='bold', fontsize=44)
    plt.tick_params(axis='both', which='major', labelsize=36)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    output_path = os.path.join(output_folder, f'{model_name}_Correlation_Heatmap.png')
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    print(f"Saved correlation heatmap at {output_path}")
    plt.close()

def plot_shap_values(model, X, feature_names, model_name, feature_importance):
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    
    # 按照特征重要性排序
    feature_order = feature_importance['Feature'].tolist()
    feature_idx = [feature_names.index(f) for f in feature_order if f in feature_names]
    
    plt.figure(figsize=(14, 12))
    shap.summary_plot(shap_values[:, feature_idx], X.iloc[:, feature_idx], 
                      plot_type="dot", feature_names=[feature_names[i] for i in feature_idx], 
                      show=False, plot_size=(14, 12), color=colors[model_name][0])
    plt.title(f'SHAP Feature Impact - {model_name}', fontweight='bold', fontsize=44)
    plt.tick_params(axis='both', which='major', labelsize=36)
    plt.tight_layout()
    output_path = os.path.join(output_folder, f'{model_name}_shap_plot.png')
    plt.savefig(output_path, dpi=600, bbox_inches='tight')
    print(f"Saved SHAP plot for {model_name} at {output_path}")
    plt.close()

# 数据预处理
df = df.apply(pd.to_numeric, errors='coerce')
df = df.fillna(0)

# 分离特征和目标变量
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
feature_names = X.columns.tolist()

print(f"\nInitial number of features: {X.shape[1]}")

# 分层抽样分割数据集（确保目标变量分布一致）
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
y_binary = (y != 0).astype(int)  # 将连续目标变量转为二分类，用于分层抽样

for train_index, val_index in sss.split(X, y_binary):
    X_train, X_test = X.iloc[train_index], X.iloc[val_index]
    y_train, y_test = y[train_index], y[val_index]

print(f"Training set size: {len(X_train)} samples")
print(f"Test set size: {len(X_test)} samples")

# 定义模型管道（标准化→特征选择→模型训练）
def create_pipeline(model):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('feature_selector', CustomFeatureSelector()),
        ('model', model)
    ])

# 优化后的模型配置（超参数搜索范围）
models = {
    'XGBoost': (create_pipeline(xgb.XGBRegressor(random_state=42)), {
        'feature_selector__k': [10, 15, 20, 25],
        'model__n_estimators': [100, 200, 300, 400],
        'model__max_depth': [3, 4, 5, 6],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__subsample': [0.8, 0.9, 1.0],
        'model__colsample_bytree': [0.8, 0.9, 1.0],
        'model__reg_alpha': [0, 0.1, 0.5],
        'model__reg_lambda': [0, 0.1, 0.5]
    }),
    'ElasticNet': (create_pipeline(ElasticNet(random_state=42)), {
        'feature_selector__k': [10, 15, 20, 25],
        'model__alpha': [0.001, 0.01, 0.1, 1],
        'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    }),
    'SVR': (create_pipeline(SVR()), {
        'feature_selector__k': [10, 15, 20, 25],
        'model__kernel': ['rbf', 'poly'],
        'model__C': [0.1, 1, 10, 100],
        'model__gamma': ['scale', 'auto'],
        'model__epsilon': [0.01, 0.1, 0.2]
    }),
    'RandomForest': (create_pipeline(RandomForestRegressor(random_state=42)), {
        'feature_selector__k': [10, 15, 20, 25],
        'model__n_estimators': [100, 200, 300, 400],
        'model__max_depth': [None, 5, 10, 15],
        'model__min_samples_split': [2, 5, 10]
    }),
    'GradientBoosting': (create_pipeline(GradientBoostingRegressor(random_state=42)), {
        'feature_selector__k': [10, 15, 20, 25],
        'model__n_estimators': [100, 200, 300, 400],
        'model__max_depth': [3, 4, 5, 6],
        'model__learning_rate': [0.01, 0.05, 0.1],
        'model__subsample': [0.8, 0.9, 1.0],
    })
}

# 训练和评估模型（贝叶斯优化超参数）
best_models = {}
best_r2 = -np.inf
best_model_name = ''
results = {}  # 存储每个模型的性能指标和预测结果

for name, (model, params) in models.items():
    print(f"\n=== Training {name} Model ===")
    # 贝叶斯超参数搜索（5折交叉验证，重复3次）
    opt = BayesSearchCV(
        model,
        params,
        cv=RepeatedKFold(n_splits=5, n_repeats=3),
        scoring='r2',  # 以R²为优化目标
        n_jobs=-1,     # 并行计算（使用所有CPU核心）
        n_iter=50,     # 搜索50组超参数组合
        random_state=42
    )
    
    opt.fit(X_train, y_train)
    best_model = opt.best_estimator_
    best_models[name] = best_model

    # 模型预测
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)
    
    # 计算性能指标
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    
    # 保存结果
    results[name] = {
        'r2_train': r2_train,
        'r2_test': r2_test,
        'mae_train': mae_train,
        'mae_test': mae_test,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred,
        'best_params': opt.best_params_
    }
    
    # 打印性能结果
    print(f"{name} Performance:")
    print(f"  Train R²: {r2_train:.3f}, Train MAE: {mae_train:.3f}")
    print(f"  Test R²: {r2_test:.3f}, Test MAE: {mae_test:.3f}")
    print(f"  Best Hyperparameters: {opt.best_params_}")
    
    # 绘制预测对比图
    plot_prediction(y_train, y_train_pred, y_test, y_test_pred, name, r2_train, r2_test)

    # 更新最优模型
    if r2_test > best_r2:
        best_r2 = r2_test
        best_model_name = name

# 对最佳模型进行深度分析
print(f"\n=== Best Model Analysis: {best_model_name} ===")
print(f"Best Model Test R²: {best_r2:.3f}")
best_model = best_models[best_model_name]
y_train_pred = results[best_model_name]['y_train_pred']
y_test_pred = results[best_model_name]['y_test_pred']

# -------------------------- 修复核心：预测数据行拼接（统一长度） --------------------------
# 训练集预测数据（含样本ID和数据来源标识）
train_pred_data = pd.DataFrame({
    'Set': 'Training',  # 标识数据来源（训练集/测试集）
    'Sample_ID': [f'Train_{i+1}' for i in range(len(y_train))],  # 唯一样本ID
    'Actual_Value': y_train.values,  # 真实值
    'Predicted_Value': y_train_pred,  # 预测值
    'Residual': y_train.values - y_train_pred  # 残差（新增，便于误差分析）
})

# 测试集预测数据（格式与训练集一致）
test_pred_data = pd.DataFrame({
    'Set': 'Test',
    'Sample_ID': [f'Test_{i+1}' for i in range(len(y_test))],
    'Actual_Value': y_test.values,
    'Predicted_Value': y_test_pred,
    'Residual': y_test.values - y_test_pred
})

# 合并训练集和测试集数据（行拼接，长度统一）
prediction_data = pd.concat([train_pred_data, test_pred_data], ignore_index=True)

# 保存预测数据到CSV（用于Origin作图）
prediction_csv_path = os.path.join(output_folder, f'{best_model_name}_Prediction_Data.csv')
prediction_data.to_csv(prediction_csv_path, index=False, encoding='utf-8')
print(f"\nSaved prediction data (with residual) to: {prediction_csv_path}")

# -------------------------- 特征重要性分析 --------------------------
# 获取特征选择器选中的特征
feature_selector = best_model.named_steps['feature_selector']
selected_feature_indices = feature_selector.selector.get_support(indices=True)
selected_feature_names = [feature_names[i] for i in selected_feature_indices]

# 计算特征重要性（不同模型的重要性指标不同）
if hasattr(best_model.named_steps['model'], 'feature_importances_'):
    # 树模型（XGBoost、RandomForest、GradientBoosting）
    importances = best_model.named_steps['model'].feature_importances_
elif hasattr(best_model.named_steps['model'], 'coef_'):
    # 线性模型（ElasticNet）：用系数绝对值表示重要性
        importances = np.abs(best_model.named_steps['model'].coef_)
else:
    # SVR等无法直接获取重要性的模型
    print(f"Cannot directly calculate feature importance for {best_model_name}, using mutual info scores instead")
    importances = feature_selector.selector.scores_[selected_feature_indices]

# 整理特征重要性数据
importance_df = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False).reset_index(drop=True)

# 保存特征重要性到CSV
importance_csv_path = os.path.join(output_folder, f'{best_model_name}_Feature_Importance.csv')
importance_df.to_csv(importance_csv_path, index=False, encoding='utf-8')
print(f"Saved feature importance data to: {importance_csv_path}")

# 绘制Top6特征重要性图
plot_feature_importance(importance_df, best_model_name)

# -------------------------- 相关性分析（Top6特征） --------------------------
top6_features = importance_df['Feature'].head(6).tolist()
# 提取训练集中Top6特征的原始数据
X_train_top6 = pd.DataFrame(X_train, columns=feature_names)[top6_features]
# 计算相关性矩阵
corr_matrix = X_train_top6.corr()

# 保存相关性矩阵到CSV
correlation_csv_path = os.path.join(output_folder, f'{best_model_name}_Top6_Features_Correlation.csv')
corr_matrix.to_csv(correlation_csv_path, encoding='utf-8')
print(f"Saved Top6 features correlation matrix to: {correlation_csv_path}")

# 绘制相关性热图
plot_correlation_heatmap(corr_matrix, best_model_name)

# -------------------------- SHAP值分析（仅树模型支持） --------------------------
if hasattr(best_model.named_steps['model'], 'feature_importances_'):
    # 提取测试集中选中的特征数据（用于SHAP解释）
    X_test_selected = pd.DataFrame(X_test, columns=feature_names)[selected_feature_names]
    # 绘制SHAP特征影响图
    plot_shap_values(
        model=best_model.named_steps['model'],
        X=X_test_selected,
        feature_names=selected_feature_names,
        model_name=best_model_name,
        feature_importance=importance_df
    )
else:
    print(f"SHAP analysis is only supported for tree-based models (XGBoost/RandomForest/GradientBoosting), skipping for {best_model_name}")

# -------------------------- 所有模型性能汇总 --------------------------
# 整理性能数据（R²、MAE）
performance_data = []
for model_name, result in results.items():
    performance_data.append({
        'Model': model_name,
        'Train_R²': result['r2_train'],
        'Test_R²': result['r2_test'],
        'Train_MAE': result['mae_train'],
        'Test_MAE': result['mae_test'],
        'Best_Feature_Count': result['best_params']['feature_selector__k']
    })

performance_df = pd.DataFrame(performance_data).sort_values('Test_R²', ascending=False)

# 保存性能汇总到CSV
performance_csv_path = os.path.join(output_folder, 'All_Models_Performance_Summary.csv')
performance_df.to_csv(performance_csv_path, index=False, encoding='utf-8')
print(f"\nSaved all models performance summary to: {performance_csv_path}")

# 打印最终汇总信息
print("\n=== Final Summary ===")
print("All generated files are saved to:", output_folder)
print("\nModel Performance Ranking (by Test R²):")
for idx, row in performance_df.iterrows():
    print(f"{row['Model']}: Test R² = {row['Test_R²']:.3f}, Test MAE = {row['Test_MAE']:.3f}")
print(f"\nBest Model: {best_model_name} (Test R² = {best_r2:.3f})")
print("All plots and data tables have been generated successfully!")